In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.integrate import quad
from scipy.special import j0, j1

# Constants for mapping
value_mapping = {
    "Pressure": {-1: 0.02, 1: 7},
    "Youngs_modulus": {-1: 2.1, 1: 25},
    "Poisson_ratio": {-1: 0.2, 1: 0.31},
    "Depth": {-1: 800, 1: 3250},
    "Radius": {-1: 400, 1: 5100},
    "Thickness": {-1: 20, 1: 676},
}

def calculate_displacement(pressure, youngs_modulus, poisson_ratio, depth, radius, thickness, r_values):
    """Calculate displacement for a given set of parameters."""
    try:
        alpha_B = 1  # Biot coefficient

        dp_res = pressure * 1e6  # Convert pressure to Pa
        E = youngs_modulus * 1e9  # Convert Young's modulus to Pa
        v = poisson_ratio
        d = depth  # Depth
        R = radius  # Radius
        h = thickness  # Thickness

        if E <= 0 or (1 - v) <= 0 or (1 - 2 * v) <= 0:
            return [np.nan] * len(r_values)

        Cm = alpha_B * ((1 + v) * (1 - 2 * v)) / ((1 - v) * E)

        def integrand(alpha, r, d, R):
            return np.exp(-alpha * d) * j1(alpha * R) * j0(alpha * r)

        def uz_formula_corrected(r):
            integral_value, _ = quad(integrand, 0, np.inf, args=(r, d, R), epsabs=1e-10, epsrel=1e-10)
            uz = 2 * Cm * (1 - v) * h * dp_res * R * integral_value
            return uz * 1000  # Convert displacement to mm

        # Calculate displacements for all r values
        uz_values = [uz_formula_corrected(r) for r in r_values]
        return uz_values

    except Exception:
        return [np.nan] * len(r_values)


def load_and_map_data(file_path):
    """Load and map the data based on predefined mappings."""
    data = pd.ExcelFile(file_path)
    sheet_data = data.parse('Sheet1')
    relevant_data = sheet_data.iloc[:, :8]  # Extract first 8 columns
    relevant_data.columns = [
        "StdOrder", "Pressure", "Youngs_modulus", "Poisson_ratio", 
        "Depth", "Radius", "Thickness", "Displacement_at_r0"
    ]


    for param, mapping in value_mapping.items():
        relevant_data[param] = relevant_data[param].map(mapping)

    return relevant_data


def filter_extreme_profiles(row, r_values, max_displacement=1000, max_std_dev=100):
    """Filter out extreme profiles based on displacement criteria."""
    displacement_values = row["Displacement_profiles"]

    # Calculate max, min displacement and standard deviation
    max_disp = np.max(displacement_values)
    min_disp = np.min(displacement_values)
    std_dev = np.std(displacement_values)

    # Check if displacement exceeds thresholds or if standard deviation is too high
    if max_disp > max_displacement or std_dev > max_std_dev or min_disp < 0:
        return False  # Exclude extreme profiles
    return True


def plot_displacement_profiles(r_values, filtered_data):
    """Plot displacement profiles for filtered data."""
    plt.figure(figsize=(10, 6))
    for idx, row in filtered_data.iterrows():
        plt.plot(r_values, row["Displacement_profiles"], color='b', alpha=0.3)  # Transparent lines

    plt.title('Displacement Profiles for All Factorial Analysis Runs, Pressure=0.02 MPa, with filter')
    plt.xlabel('Radius (m)')
    plt.ylabel('Displacement (mm)')
    plt.legend(['Displacement Profile'], loc='upper right', frameon=False)
    plt.tight_layout()
    plt.show()


def main(file_path='DOE_FULL.xlsx', max_displacement=1000, max_std_dev=100):
    """Main function to load data, calculate displacements, filter and plot profiles."""

    relevant_data = load_and_map_data(file_path)


    r_values = np.linspace(0, 15100, 250)


    relevant_data["Displacement_profiles"] = relevant_data.apply(
        lambda row: calculate_displacement(
            row["Pressure"], row["Youngs_modulus"], row["Poisson_ratio"], 
            row["Depth"], row["Radius"], row["Thickness"], r_values
        ),
        axis=1
    )

    # Filter extreme profiles based on displacement criteria
    filtered_combinations = []
    filtered_data = relevant_data[relevant_data.apply(
        lambda row: filter_extreme_profiles(row, r_values), axis=1)]


    plot_displacement_profiles(r_values, filtered_data)


    if filtered_combinations:
        print("Filtered Factor Combinations:")
        for combination in filtered_combinations:
            print(combination)
    else:
        print("No factor combinations were filtered.")

    # Save filtered results
    output_file = 'AUC_filter_0.02.xlsx'
    filtered_data.to_excel(output_file, index=False)
    print(f"Results have been saved to {output_file}")


if __name__ == "__main__":
    main()
